In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anuvaad-03/anuvaad_0.3/translator/fingerprint.pb
/kaggle/input/anuvaad-03/anuvaad_0.3/translator/saved_model.pb
/kaggle/input/anuvaad-03/anuvaad_0.3/translator/variables/variables.index
/kaggle/input/anuvaad-03/anuvaad_0.3/translator/variables/variables.data-00000-of-00001
/kaggle/input/final-20-15e/translator/fingerprint.pb
/kaggle/input/final-20-15e/translator/saved_model.pb
/kaggle/input/final-20-15e/translator/variables/variables.index
/kaggle/input/final-20-15e/translator/variables/variables.data-00000-of-00001
/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator/fingerprint.pb
/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator/saved_model.pb
/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator/variables/variables.index
/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator/variables/variables.data-00000-of-00001
/kaggle/input/final-20-015-mixed-00409/translator1/fingerprint.pb
/kaggle/input/final-20-015-mixed-00409/transla

In [2]:
from nltk.translate.bleu_score import sentence_bleu

In [3]:
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.79.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [4]:
import tensorflow as tf
import tensorflow_text as tf_text

In [5]:
!pip install Metaphone
from metaphone import doublemetaphone

  Preparing metadata (setup.py) ... done
  Created wheel for Metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13919 sha256=f9440d3d13e5b56e8681cf2ab02e59052e3012816e459ee8d2f75357bb05cbb7
  Stored in directory: /root/.cache/pip/wheels/3a/2c/57/cb7d38aa6c80a870a0341ccf4265c2921fb55b1f7f4b76b46b
Successfully built Metaphone


In [6]:
import pickle
with open('/kaggle/input/hin-vocab/vocab_data.pkl', 'rb') as fp:
    hin_vocab = pickle.load(fp)
vocab_keys=[l for l in hin_vocab]
len(hin_vocab),len(vocab_keys)

(26214, 26214)

In [7]:
import re
re.sub("[.!?\\-\'\"]", "", "    kaise ho tum viase .?,KSDFaha   ").lower().strip()

'kaise ho tum viase ,ksdfaha'

In [8]:
def t_text(line):
    line=re.sub("[.!?\\-\'\"]", "",line).lower().strip()
    string=''
    for j in line.split(' '):
        if doublemetaphone(j)[0]+'*'+doublemetaphone(j[::-1])[0]+'*'+j[:2]+'*'+j[len(j)-1:] in vocab_keys:
            string=string+list(hin_vocab[doublemetaphone(j)[0]+'*'+doublemetaphone(j[::-1])[0]+'*'+j[:2]+'*'+j[len(j)-1:]])[0]+' '
        else:
            string=string+j+' '
    return string.lower().strip()

In [9]:
import pandas as pd
data_val=pd.read_csv('/kaggle/input/validate-bleu/validation.tsv',sep='\t')
data_val=data_val[['cs_query','en_query']]
data_val=data_val.rename(columns={'cs_query':'Sentence','en_query':'trr'})
data_val['trr']=data_val['trr'].apply(lambda x: x.lower().strip())
data_val

,Sentence,trr
0,mere timer ko roko,pause my timer .
1,Please Liz ke liye video message banaye,please make video message for liz
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday
3,Library events Cedar Park me hai,library events in cedar park
4,Please timer ko rokey,please pause timer
...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,i ' d like you to snooze my alarm for 10 minut...
1387,please Tupac play kare,please play tupac
1388,Rhonda ko message kare aur July me Vacation Bi...,message rhonda ask her details about vacation ...


In [10]:
'''three_input_text = tf.constant([t_text('sabse best friend'), 
                                t_text('That toh I know'),
                                t_text('She was bhunnoing the masalas jub phone ki ghuntee bugee'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhara papa ka naam kya hai'),
])

result = reloaded.translate(input_text=three_input_text)

for tr in result['text']:
  print(tr.numpy().decode())'''

"three_input_text = tf.constant([t_text('sabse best friend'), \n                                t_text('That toh I know'),\n                                t_text('She was bhunnoing the masalas jub phone ki ghuntee bugee'),\n                                t_text('tum batao'),\n                                t_text('kya karna hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhara papa ka naam kya hai'),\n])\n\nresult = reloaded.translate(input_text=three_input_text)\n\nfor tr in result['text']:\n  print(tr.numpy().decode())"

In [11]:
import numpy

In [12]:
def cal_bleu(x,y):
    return sentence_bleu([x.split()],y.split())

In [70]:
reloaded = tf.saved_model.load('/kaggle/input/transl-last-hour/translator1')
data_val['model_output1']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score1']=data_val[['trr','model_output1']].apply(lambda x: cal_bleu(*x),axis=1)
#data_val['bleu_score_inv1']=data_val[['trr','model_output1']].apply(lambda x: cal_bleu_inv(*x),axis=1)

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [71]:
data_val

,Sentence,trr,model_output1,bleu_score1
0,mere timer ko roko,pause my timer .,.,0.049787
1,Please Liz ke liye video message banaye,please make video message for liz,"please end the roads ? , , , ek bhi where shou...",0.500000
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday,will the roads be between pm and pm on tuesday...,0.326497
3,Library events Cedar Park me hai,library events in cedar park,library events cedar park performing,0.795271
4,Please timer ko rokey,please pause timer,please pause the timer .,0.622333
...,...,...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer,fireworks both have electricity summer,0.668740
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,i ' d like you to snooze my alarm for 10 minut...,"? poor man , the only one sunny cage on the ot...",0.000000
1387,please Tupac play kare,please play tupac,please play tupac on advance .,0.472871
1388,Rhonda ko message kare aur July me Vacation Bi...,message rhonda ask her details about vacation ...,ask shortest letting kids know about the july ...,0.289178


In [72]:
sentence_bleu(["i have eaten".split()], "i have eaten the food".split() )

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.5623413251903491

In [73]:
data_val['bleu_score1'].mean()

0.49312823182317694

In [74]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-015-mixed-00409/translator1')
data_val['model_output2']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score2']=data_val[['trr','model_output2']].apply(lambda x: cal_bleu(*x),axis=1)


/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [75]:
data_val['bleu_score2'].mean()

0.5189571164133745

In [76]:
a=['please pause timer','please pause timer']
print(a[0]==a[1])
sentence_bleu(list(a[0].split()),a[1].split())

True


0

In [77]:
a=data_val[['trr','model_output1']].values[4]
print(a)
print(a[0]==a[1])
print(a[0])
print(a[1])
sentence_bleu(a[0].split(),a[1].split())

['please pause timer' 'please pause the timer .']
False
please pause timer
please pause the timer .


0

In [78]:
from statistics import mean

In [79]:
l1=[]
for i in data_val[['trr','model_output1']].values:
    l1.append(sentence_bleu(i[0].split(),i[1].split()))
mean(l1)

0.20322533139445562

In [80]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-02-15e/translator1')
data_val['model_output3']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score3']=data_val[['trr','model_output3']].apply(lambda x: cal_bleu(*x),axis=1)

In [81]:
data_val['bleu_score3'].mean()

0.49492970947981707

In [82]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-01-15e/translator1')
data_val['model_output4']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score4']=data_val[['trr','model_output4']].apply(lambda x: cal_bleu(*x),axis=1)


In [83]:
data_val['bleu_score4'].mean()

0.5008011711980445

In [84]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-15e/translator')
data_val['model_output5']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score5']=data_val[['trr','model_output5']].apply(lambda x: cal_bleu(*x),axis=1)


In [85]:
data_val['bleu_score5'].mean()

0.5027574531278632

In [86]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-22e/translator')
data_val['model_output6']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score6']=data_val[['trr','model_output6']].apply(lambda x: cal_bleu(*x),axis=1)


In [87]:
data_val['bleu_score6'].mean()

0.4844805117457623

In [88]:
reloaded = tf.saved_model.load('/kaggle/input/translator1/translator1')
data_val['model_output7']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score7']=data_val[['trr','model_output7']].apply(lambda x: cal_bleu(*x),axis=1)


In [89]:
data_val['bleu_score7'].mean()

0.49739818722818113

In [90]:
reloaded = tf.saved_model.load('/kaggle/input/translator-2/translator1')
data_val['model_output8']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score8']=data_val[['trr','model_output8']].apply(lambda x: cal_bleu(*x),axis=1)


In [91]:
data_val['bleu_score8'].mean()

0.5050316530517747

In [92]:
reloaded = tf.saved_model.load('/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator')
reloaded.tf_translate(
                tf.constant([
                t_text('tumhara naam kya hai')
                        ]))['text'][0].numpy().decode()

'. . . . what is the difference ? ?'

In [93]:
reloaded = tf.saved_model.load('/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator')
data_val['model_output9']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score9']=data_val[['trr','model_output9']].apply(lambda x: cal_bleu(*x),axis=1)


In [94]:
data_val['bleu_score9'].mean()

0.43506236947501886

In [95]:
reloaded = tf.saved_model.load('/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator')

three_input_text = tf.constant([#'sabse best friend', 
                                #'That toh I know',
                                t_text('mera naam rohan hai'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
])

result = reloaded.tf_translate(three_input_text)
print(result['text'])
for tr in result['text']:
  print(tr.numpy().decode())

tf.Tensor(
[b'hahahahhah . . you have already gone . .' b'you made a g'
 b'what is the bomb'], shape=(3,), dtype=string)
hahahahhah . . you have already gone . .
you made a g
what is the bomb


In [96]:
reloaded = tf.saved_model.load('/kaggle/input/file-name-20-02/file_name_20_0.2/translator')
data_val['model_output10']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score10']=data_val[['trr','model_output10']].apply(lambda x: cal_bleu(*x),axis=1)


In [97]:
data_val['bleu_score10'].mean()

0.3849685106377024

In [98]:
reloaded = tf.saved_model.load('/kaggle/input/anuvaad-03/anuvaad_0.3/translator')
data_val['model_output11']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score11']=data_val[['trr','model_output11']].apply(lambda x: cal_bleu(*x),axis=1)


In [99]:
data_val['bleu_score11'].mean()

0.23454356386202788

In [105]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-cv-20-15-14e/translator1')
data_val['model_output12']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score12']=data_val[['trr','model_output12']].apply(lambda x: cal_bleu(*x),axis=1)


/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [106]:
data_val['bleu_score12'].mean()

0.4927455867210749

In [13]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-40k-mixed-batch/translator1')
data_val['model_output13']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score13']=data_val[['trr','model_output13']].apply(lambda x: cal_bleu(*x),axis=1)


/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [14]:
data_val['bleu_score13'].mean()

0.5150563926712219

In [15]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-40k-mixed-batch-v2/translator1')
data_val['model_output14']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score14']=data_val[['trr','model_output14']].apply(lambda x: cal_bleu(*x),axis=1)


In [16]:
data_val['bleu_score14'].mean()

0.5206490601193727

In [17]:
data_val

,Sentence,trr,model_output13,bleu_score13,model_output14,bleu_score14
0,mere timer ko roko,pause my timer .,my timer is completely baked,0.562341,my timer is completely blow,0.562341
1,Please Liz ke liye video message banaye,please make video message for liz,please the reminds that kulfi is hunger .,0.594604,"please the fruit of liz , goat lentils . .",0.668740
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday,create a alarm that addysen at pm and pm on tu...,0.367206,create a alarm reminding me to make an alarm a...,0.426108
3,Library events Cedar Park me hai,library events in cedar park,library events mountain garden here,0.562341,the library events in half past . it is roasts .,0.279016
4,Please timer ko rokey,please pause timer,please pause timer needed,0.707107,please pause timer less,0.707107
...,...,...,...,...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer,the concerts for the deardorff summer blast,0.731110,concerts by oingo summer laddu,0.795271
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,i ' d like you to snooze my alarm for 10 minut...,my poverty is still under a previous alarm to ...,0.744782,my lumber . . . . you want my own alarm to you...,0.716233
1387,please Tupac play kare,please play tupac,please play tupac,1.000000,please play tupac,1.000000
1388,Rhonda ko message kare aur July me Vacation Bi...,message rhonda ask her details about vacation ...,rhonda he american glen like her vacation in t...,0.707107,rhonda video live and july in maa chat with st...,0.392815


In [108]:
'ok'

'ok'

In [109]:
data_val.to_csv('bleu_score.csv', index=False)